# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Müller  ->  J. Müller-Horn  |  ['J. Müller']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 68 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.11155


extracting tarball to tmp_2503.11155...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.11451


extracting tarball to tmp_2503.11451... done.


Retrieving document from  https://arxiv.org/e-print/2503.11621


extracting tarball to tmp_2503.11621... done.


Found 55 bibliographic references in tmp_2503.11621/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.11621-b31b1b.svg)](https://arxiv.org/abs/2503.11621) | **Euclid preparation. BAO analysis of photometric galaxy clustering in configuration space**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-17*|
|*Comments*| *18 pages, 12 figures, submitted to A&A*|
|**Abstract**|            With about 1.5 billion galaxies expected to be observed, the very large number of objects in the \textit{Euclid}\xspace photometric survey will allow for precise studies of galaxy clustering from a single survey, over a large range of redshifts $0.2 < z < 2.5$. In this work, we use photometric redshifts to extract the baryon acoustic oscillation signal (BAO) from the Flagship galaxy mock catalogue with a tomographic approach to constrain the evolution of the Universe and infer its cosmological parameters. We measure the two-point angular correlation function in 13 redshift bins. A template-fitting approach is applied to the measurement to extract the shift of the BAO peak through the transverse Alcock--Paczynski parameter $\alpha$. A joint analysis of all redshift bins is performed to constrain $\alpha$ at the effective redshift $z_\mathrm{eff}=0.77$ with MCMC and profile likelihood techniques. We also extract one $\alpha_i$ parameter per redshift bin to quantify its evolution as a function of time. From these 13 $\alpha_i$, which are directly proportional to the ratio $D_\mathrm{A}/\,r_\mathrm{s,\,drag}$, we constrain $h$, $\Omega_\mathrm{b}$, and $\Omega_\mathrm{cdm}$. From the joint analysis, we constrain $\alpha(z_\mathrm{eff}=0.77)=1.0011^{+0.0078}_{-0.0079}$, which represents a three-fold improvement over current constraints from the Dark Energy Survey. As expected, the constraining power in the analysis of each redshift bin is lower, with an uncertainty ranging from $\pm\,0.13$ to $\pm\,0.024$. From these results, we constrain $h$ at 0.45 %, $\Omega_\mathrm{b}$ at 0.91 %, and $\Omega_\mathrm{cdm}$ at 7.7 %. We quantify the influence of analysis choices like the template, scale cuts, redshift bins, and systematic effects like redshift-space distortions over our constraints both at the level of the extracted $\alpha_i$ parameters and at the level of cosmological inference.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.11155-b31b1b.svg)](https://arxiv.org/abs/2503.11155) | **Surface brightness-color relations for red giant branch stars using asteroseismic radii and Gaia distances (the ARD method)**  |
|| J. Xiong, et al. -- incl., <mark>J. Li</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-17*|
|*Comments*| **|
|**Abstract**|            Aims. Asteroseismic radius and Gaia distance (ARD) method has been proposed to establish the SBCRs for late-type stars. Methods. We select Kepler RGB stars with high-precision asteroseismic radii (uncertainties < 1%) and cross-match them with 2MASS, APASS, and Gaia to obtain Johnson-B, Johnson-V, G, J, H, and Ks-band photometric data. After applying selection criteria, we obtain 626 RGB stars to build the SBCR. Among these, 100 RGBs are used as independent validation for the distance, and the remaining samples are used to fit the SBCR. Results. First, using 526 targets with asteroseismic radii and Gaia distances, nine SBCRs are proposed based on 2MASS (J, H, Ks), APASS (Johnson-B, Johnson-V), and Gaia (G) photometry. The average rms scatter in these relations is 0.075 mag, which corresponds to an uncertainty of approximately 3.5% in distance. These relations are further validated using 100 independent samples with Gaia distances, showing no bias, with a dispersion of approximately 3%. Compared to interferometric measurements, a systematic underestimation of 2.3% was observed, and the discrepancy decreases as the angular diameter increases. Additionally, the distances of eclipsing binaries in the Large Magellanic Cloud and Small Magellanic Cloud obtained using our SBCRs are generally consistent with those measured in the literature, with a dispersion of 1% and a slight overestimation of 1% to 2.5%. Conclusions. The ARD method capitalizes on two key advantages for precise stellar distance determination: a statistically robust sample of homogeneous RGB stars with low observational costs, and independent distance verification through Gaia data. Such SBCRs can be further calibrated and expanded more efficiently and effectively.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.11451-b31b1b.svg)](https://arxiv.org/abs/2503.11451) | **Oscillations of red giant stars with magnetic damping in the core. I. Dissipation of mode energy in dipole-like magnetic fields**  |
|| <mark>J. Müller</mark>, Q. Coppée, S. Hekker |
|*Appeared on*| *2025-03-17*|
|*Comments*| *accepted by A&A, 20 pages, 19 figures, 3 tables*|
|**Abstract**|            Strong magnetic fields in the core of red-giant branch stars are expected to suppress the amplitudes of the multipole modes. This occurs when the strength of the internal magnetic field approaches the critical field strength, at which the magnetic forces become comparable to the buoyancy. We performed Hamiltonian ray tracing simulations of magneto-gravity waves to investigate the suppression of the multipole modes in the presence of an internal dipole-like magnetic field. We took into account different stellar masses, metallicities, and ages, as well as various oscillation frequencies and spherical degrees. In particular, we estimated the trapped fraction, a measure of multipole mode suppression, which quantifies the fraction of mode energy in the core that is dissipated by the interaction with the magnetic field. Our results indicate that the trapped fraction can be described by a simple expression, which smoothly connects the regime without multipole mode suppression with the regime with complete suppression of the multipole modes. Crucially, the trapped fraction depends only on the ratio between the strength of the internal magnetic field and the critical field strength. Therefore, our expression for the trapped fraction provides a flexible tool that can be used, for example, to estimate the amount of multipole mode suppression as a star ascends the red-giant branch or to investigate the onset of the suppression in observed power spectral densities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.11621/./fig_flagship_n_z_distribution.png', 'tmp_2503.11621/./fig_cosmo_bins_robustness.png', 'tmp_2503.11621/./fig_2pcf_bao_1.png']
copying  tmp_2503.11621/./fig_flagship_n_z_distribution.png to _build/html/
copying  tmp_2503.11621/./fig_cosmo_bins_robustness.png to _build/html/
copying  tmp_2503.11621/./fig_2pcf_bao_1.png to _build/html/
exported in  _build/html/2503.11621.md
    + _build/html/tmp_2503.11621/./fig_flagship_n_z_distribution.png
    + _build/html/tmp_2503.11621/./fig_cosmo_bins_robustness.png
    + _build/html/tmp_2503.11621/./fig_2pcf_bao_1.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation.: BAO analysis of photometric galaxy clustering in configuration space

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.11621-b31b1b.svg)](https://arxiv.org/abs/2503.11621)<mark>Appeared on: 2025-03-17</mark> -  _18 pages, 12 figures, submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** With about 1.5 billion galaxies expected to be observed, the very large number of objects in the $\Euclid$ photometric survey will allow for precise studies of galaxy clustering from a single survey, over a large range of redshifts $0.2 < z < 2.5$ . In this work, we use photometric redshifts ( $\zph$ ) to extract the baryon acoustic oscillation signal (BAO) from the Flagship galaxy mock catalogue with a tomographic approach to constrain the evolution of the Universe and infer its cosmological parameters. We measure the two-point angular correlation function in 13 redshift bins. A template-fitting approach is applied to the measurement to extract the shift of the BAO peak through the transverse Alcock--Paczynski parameter $\alpha$ . A joint analysis of all redshift bins is performed to constrain $\alpha$ at the effective redshift $z_\mathrm{eff}=0.77$ with Markov Chain Monte-Carlo and profile likelihood techniques. We also extract one $\alpha_i$ parameter per redshift bin to quantify its evolution as a function of time. From these 13 $\alpha_i$ , which are directly proportional to the ratio $D_\mathrm{A}/ r_\mathrm{s, drag}$ , we constrain the reduced Hubble constant $h$ , the baryon density parameter $\Omb$ , and the cold dark matter density parameter $\Omcdm$ . From the joint analysis, we constrain $\alpha(z_\mathrm{eff}=0.77)=1.0011^{+0.0078}_{-0.0079}$ at the 68 \% confidence level, which represents a three-fold improvement over current constraints from the Dark Energy Survey (uncertainty of $\pm 0.023$ at $z_\mathrm{eff}=0.85$ with the same observable). As expected, the constraining power in the analysis of each redshift bin is lower, with an uncertainty ranging from $\pm 0.13$ to $\pm 0.024$ . From these results, we constrain $h$ at 0.45 \% , $\Omb$ at 0.91 \% , and $\Omcdm$ at 7.7 \% . We quantify the influence of analysis choices like the template, scale cuts, redshift bins, and systematic effects like redshift-space distortions over our constraints both at the level of the extracted $\alpha_i$ parameters and at the level of cosmological inference.

</div>

<div id="div_fig1">

<img src="tmp_2503.11621/./fig_flagship_n_z_distribution.png" alt="Fig1" width="100%"/>

**Figure 1. -** True redshift distribution of the galaxies from Flagship 2.1.10 selected in 13 equipopulated photometric redshift bins. (*fig:flagship_nz*)

</div>
<div id="div_fig2">

<img src="tmp_2503.11621/./fig_cosmo_bins_robustness.png" alt="Fig6" width="100%"/>

**Figure 6. -** Comparison of cosmological parameters obtained when one or several redshift bins are removed from the analysis. The fiducial values of the simulation are highlighted as a vertical dashed line. The baseline including all bins is shown at the top as a reference. The results in blue and orange are respectively obtained from the $\alpha_{i}$ extracted by template fitting of the $w(\theta)$ measured on Flagship and a noise-free synthetic $w(\theta)$.  (*fig:cosmo_bins_robustness*)

</div>
<div id="div_fig3">

<img src="tmp_2503.11621/./fig_2pcf_bao_1.png" alt="Fig9" width="100%"/>

**Figure 9. -** Two-point angular correlation function measured on the Flagship simulation in 13 redshift bins. The errors come from the analytical covariance presented in Sect. \ref{sc:covariance}. The orange curve is the correlation function computed using the template from Eq.\eqref{eq:template_BAO} evaluated with the parameters inferred from MCMC in each redshift bin. Scale cuts are shown as grey bands and are defined as $\thmin=\ang{0.6}$, $\thmax=\thbao+\ang{2.5}$ where $\thbao$ is the expected position of the BAO peak in the fiducial cosmology. (*fig:flagship_2pcf*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.11621"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

365  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
